In [1]:
import torch
import numpy as np

In [6]:
from transformers import pipeline

pipeline = pipeline(task="text-generation", model="google/gemma-3-270m")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8c0d26b1-2345-4fc6-834f-20da2853a30b)')' thrown while requesting HEAD https://huggingface.co/google/gemma-3-270m/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Device set to use mps:0


In [7]:
pipeline("who are you?")

[{'generated_text': 'who are you?If you\'ve been following me on social media, you probably know that I am a big fan of the <b><i>Marvel</i></b> comics.  I love the characters and I love the plots.  I just love the world of comics.  I also love the idea of comics being so different from the "other" comic, but at the same time, I love the idea of it being the "other" comic.  I love the idea that comics are a medium that we use to expand our knowledge of the world, and that we can share that knowledge.\n\nI recently came across a story that I liked.  I was going to tell it to a friend, but I didn\'t know how.  I\'m not a comic book person, but I think that I would be interested in reading this story, so I read it.  I read it in a couple of days, but I had no idea what to expect.  I was just really intrigued by the story.\n\nI have a really long story to tell.  It\'s about my dad, who was in prison and was trying to figure out how he was going to pay for the cost of his freedom.  I was in

In [8]:
from datasets import load_dataset

dataset = load_dataset("collabllm/collabllm-multiturn-medium-large", split="train")

2026-01-21 21:30:50,393 - urllib3.connectionpool - DEBUG - Resetting dropped connection: huggingface.co
2026-01-21 21:30:50,604 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /datasets/collabllm/collabllm-multiturn-medium-large/resolve/main/README.md HTTP/1.1" 307 0
2026-01-21 21:30:50,648 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /api/resolve-cache/datasets/collabllm/collabllm-multiturn-medium-large/b921e19eac426e5c7788c38ebea9ddb760999d0c/README.md HTTP/1.1" 200 0
2026-01-21 21:30:50,764 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /datasets/collabllm/collabllm-multiturn-medium-large/resolve/b921e19eac426e5c7788c38ebea9ddb760999d0c/collabllm-multiturn-medium-large.py HTTP/1.1" 404 0
2026-01-21 21:30:50,768 - urllib3.connectionpool - DEBUG - Resetting dropped connection: s3.amazonaws.com
2026-01-21 21:30:51,072 - urllib3.connectionpool - DEBUG - https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/da

In [5]:
dataset

Dataset({
    features: ['conv_id', 'prompt', 'completion', 'score', 'turn_id', 'single_turn_prompt', 'single_turn_completion', 'single_turn_metadata', 'sessions', 'rewards'],
    num_rows: 4606
})

In [9]:
from dataset_utils import multiturn_dataset_to_sft
dataset_clean = multiturn_dataset_to_sft(dataset, eval_ratio=0.1, lower_bound_metric=0.1)

In [10]:
dataset_clean

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 447
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 49
    })
})

In [1]:
import logging

# Basic configuration - logs to console
logging.basicConfig(
    level=logging.DEBUG,  # Show all levels
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)
logger.debug("Now this will appear!")

2026-01-21 21:39:04,523 - __main__ - DEBUG - Now this will appear!


In [4]:
from train_sft import load_and_train_sft
load_and_train_sft(
    hf_model_path="google/gemma-3-270m-it",
    hf_dataset_path="collabllm/collabllm-multiturn-medium-large")


2026-01-21 21:44:52,492 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /datasets/collabllm/collabllm-multiturn-medium-large/resolve/main/README.md HTTP/1.1" 307 0
2026-01-21 21:44:52,534 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /api/resolve-cache/datasets/collabllm/collabllm-multiturn-medium-large/b921e19eac426e5c7788c38ebea9ddb760999d0c/README.md HTTP/1.1" 200 0
2026-01-21 21:44:52,658 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /datasets/collabllm/collabllm-multiturn-medium-large/resolve/b921e19eac426e5c7788c38ebea9ddb760999d0c/collabllm-multiturn-medium-large.py HTTP/1.1" 404 0
2026-01-21 21:44:52,664 - urllib3.connectionpool - DEBUG - Resetting dropped connection: s3.amazonaws.com
2026-01-21 21:44:52,961 - urllib3.connectionpool - DEBUG - https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/collabllm/collabllm-multiturn-medium-large/collabllm/collabllm-multiturn-medium-large.py HTTP/1.1

StopIteration: 

In [19]:
def preprocess_function(example):
    return {
        "prompt": example["prompt"],
        "completion": [{"role" : "assistant", "content": example["completion"]}]
    }

dataset = dataset.map(preprocess_function, remove_columns=dataset.column_names)
print(next(iter(dataset)))

Map:   0%|          | 0/4606 [00:00<?, ? examples/s]

{'prompt': [{'content': 'Hey, can you help me create a promotional document for our custom brand jewelry and accessories? I want it to focus on the quality of our products and how popular they are globally.', 'role': 'user'}], 'completion': [{'content': "To create an effective promotional document for your custom brand jewelry and accessories, it's important to consider a few details. Could you let me know the format you have in mind? For example, are you thinking of a brochure, a digital flyer, or a more detailed catalog? Additionally, who is your target audience for this document? Knowing whether it's for retail partners, end consumers, or a different group will help tailor the content. \n\nAs for content, would you like to include specific product highlights, customer testimonials, or any data about your global popularity? Let me know what you envision, and I can help draft a structure or even some sample content for you. Feel free to share any additional preferences or ideas you ha

In [23]:
from trl import SFTTrainer
from datasets import load_dataset

trainer = SFTTrainer(
    model="Qwen/Qwen3-0.6B",
    train_dataset=load_dataset("trl-lib/Capybara", split="train"))
trainer.train()

README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/37.2M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/482k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15806 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/15806 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/15806 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/Users/robertgordan/Projects/CollablLLM-repro/.venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


RuntimeError: MPS backend out of memory (MPS allocated: 15.84 GiB, other allocations: 513.94 MiB, max allowed: 18.13 GiB). Tried to allocate 4.64 GiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).